In [28]:
import pandas as pd
import folium
import random
from ipywidgets import interact, IntSlider, Output
from pyproj import Transformer

# Lecture du fichier xlsx
df_berce = pd.read_excel('./data/berce_du_caucase.xlsx')

# On garde uniquement les colonnes latitudes, longitude, altitude et date
df_berce = df_berce[['decimalLatitude', 'decimalLongitude', 'elevation', 'year']]

# On renomme les colonnes
df_berce.columns = ['latitude', 'longitude', 'elevation', 'year']

In [29]:
#Remove des lignes avec des valeurs manquantes
df_berce = df_berce.dropna()

#Remove des lignes en dessous de l'année 2000
df_berce = df_berce[df_berce.year >= 2000]

In [30]:
# Initialisation des transformateurs de coordonnées
transformer_to_lv95 = Transformer.from_crs("EPSG:4326", "EPSG:2056", always_xy=True)
transformer_to_wgs84 = Transformer.from_crs("EPSG:2056", "EPSG:4326", always_xy=True)

# Fonction pour convertir latitude/longitude en coordonnées LV95 et les placer dans des carrés de 1 km²
def convert_to_grid(row):
    x, y = transformer_to_lv95.transform(row['longitude'], row['latitude'])
    # Arrondir les coordonnées au km pour obtenir les centres des carrés de 1 km²
    row['grid_x'] = round(x, -3)
    row['grid_y'] = round(y, -3)
    return row

# Application de la conversion sur chaque ligne
df_berce = df_berce.apply(convert_to_grid, axis=1)

In [31]:
# Créer un widget Output pour afficher la carte
output = Output()

# Fonction pour générer une couleur aléatoire
def random_color():
    return f'#{random.randint(0, 255):02x}{random.randint(0, 255):02x}{random.randint(0, 255):02x}'

# Fonction pour mettre à jour la carte en fonction de l'année sélectionnée
def update_map(selected_year):
    # Effacer la carte
    m = folium.Map(location=[46.8182, 8.2275], zoom_start=8)

    # Filtrer les données en fonction de l'année sélectionnée
    filtered_data = df_berce[df_berce['year'] == selected_year]

    # Grouper par les coordonnées de grille pour éviter des points multiples au même endroit
    grouped_data = filtered_data.groupby(['grid_x', 'grid_y']).size().reset_index(name='count')

    # Ajouter des carrés pour chaque groupe de points
    for index, row in grouped_data.iterrows():
        # Calculer les coins du carré de 1 km en mètres (LV95)
        top_left = (row['grid_x'] - 500, row['grid_y'] + 500)
        bottom_right = (row['grid_x'] + 500, row['grid_y'] - 500)
        
        # Convertir les coins du carré en latitude/longitude (WGS84)
        top_left_wgs84 = transformer_to_wgs84.transform(top_left[0], top_left[1])
        bottom_right_wgs84 = transformer_to_wgs84.transform(bottom_right[0], bottom_right[1])

        folium.Rectangle(
            bounds=[[top_left_wgs84[1],top_left_wgs84[0]], [bottom_right_wgs84[1], bottom_right_wgs84[0]]],
            color='transparent',
            fill=True,
            fill_color='red',
            fill_opacity=0.5,
            popup=f"Nombre d'occurrences: {row['count']}, Année: {selected_year}"
        ).add_to(m)

    # Afficher la carte mise à jour
    with output:
        output.clear_output(wait=True)
        display(m)

# Créer un slider pour la sélection des années
year_slider = IntSlider(min=df_berce['year'].min(), max=df_berce['year'].max(), step=1, value=df_berce['year'].min())
interact(update_map, selected_year=year_slider)

# Afficher la carte initiale avec la première année
update_map(df_berce['year'].min())  # Afficher les points de la première année au départ
output


interactive(children=(IntSlider(value=2000, description='selected_year', max=2024, min=2000), Output()), _dom_…

Output()

In [35]:
df_berce.head(3000)

,latitude,longitude,elevation,year,grid_x,grid_y
410,46.51569,6.65419,465.5,2000.0,2540000.0,1152000.0
411,47.03720,8.58210,448.0,2000.0,2687000.0,1210000.0
412,47.25636,8.82000,495.0,2000.0,2705000.0,1235000.0
413,47.40252,9.61768,410.0,2000.0,2764000.0,1252000.0
414,47.41010,8.48949,518.5,2000.0,2679000.0,1252000.0
...,...,...,...,...,...,...
3407,47.39347,8.76562,568.5,2007.0,2700000.0,1250000.0
3408,47.33207,8.72269,440.0,2007.0,2697000.0,1243000.0
3409,47.26103,8.51111,658.5,2007.0,2681000.0,1235000.0
3410,47.24766,8.65148,408.5,2007.0,2692000.0,1234000.0
